In [21]:
import moleculegraph
import numpy as np
import toml
from jinja2 import Template
import os, shutil
from scipy.constants import Avogadro

In [22]:
# Reftor is the torsion under investigation. For this torsion the 1.4 Mie pair interactions are evaluated

#Special bond flag for molecules with two local dipols. 
#Activates the 1.4 Mie pair interaction aswell as every coulomb interaction within the local dipols

group = "diols"
name   = "1.2-Ethanediol"
molecule_graphstring = "[cH_alcohol][OH_alcohol][CH2_alcohol][CH2_alcohol][OH_alcohol][cH_alcohol]"
special_bonds = True
reftor = [1,2,3,4]

temperatures =  [273.15, 298.15, 323.15, 348.15, 373.15, 473.15, 473.15, 422.15, 353.15, 473.15]
pressures =  [1.01325, 1.01325, 1.01325, 1.01325, 1.01325,0.98, 1961.0, 1.01325, 0.98, 490.0]
densities    = [ 1006.2144131928883, 1009.6886790271717, 1088.2042967480336, 1073.2778160124858, 1054.5074018135765,1080.06, 975.5, 1015.0, 1077.0, 1011.0 ]
residual_entropies =   [-3.4400223030179387, -3.2245172974062903, -3.0265867784697806, -2.8432458591441176, -2.6720698447512468,-0.01144187810653746, -2.345951998230561, -2.3645613962590213, -2.8081016773486502, -2.161140782190686]

temperatures =  [310.0, 350.0, 370.0, 400.77696579463674, 551.2809655872711, 655.9177690994577, 551.2809655872711]
pressures =  [10.0, 10.0, 10.0, 10.0, 10.0, 87.9286846373881, 5.972267527682755]
densities =   [1075.3316421595096, 1070.1194631639632, 1052.741519351984, 1025.7610959364176, 870.3084413461762, 718.6392901310196, 8.625775979433051]
residual_entropies =   [-3.129984971865495, -2.83149700226252, -2.6943856916006537, -2.49611396171107, -1.6646837140000137, -1.158343280563413, -0.03099161847076625]

#name = "1.3-Propanediol"
#molecule_graphstring =  "[cH_alcohol][OH_alcohol][CH2_alcohol][CH2_alkane][CH2_alcohol][OH_alcohol][cH_alcohol]"
#special_bonds = True
#reftor = [1,2,3,4]
#temperatures =  [350.0, 464.0009754457082, 571.6297328794227, 684.9910715715924, 464.0009754457082, 298.15, 363.15, 333.15, 313.15, 423.15]
#pressures =  [10.0, 10.0, 11.19552321655953, 62.82991430404119, 0.5052675027027178, 1.01325, 1.01325, 1.01325, 1.01325, 1.01325]
#densities =   [950.9946324588739, 916.3550048081461, 806.0880297926766, 591.9047157857249, 1.0078418649161922, 1049.0007231381508, 1005.5413477313526, 1027.873415283634, 1039.5518546254749, 962.3825120229059]
#residual_entropies =   [-5.179063825004786, -3.884297868753589, -2.589531912502393, -1.2947659562511964, -0.014364290583759065,-5.715333998793482, -5.036067737809686, -5.354324517724892, -5.561430959665683, -4.363622717138793]

########################

molecule = moleculegraph.molecule(molecule_graphstring)

nmols = 500
#nmols = 20

if special_bonds:
    # If a torsion is twice in one molecule, then evaluate the mie interaction for every 1.4 pair in this molecule
    reftor_name = [k for k,l in zip(molecule.torsion_keys,molecule.torsion_list) if list(l) == reftor ][0]
    reftor_list = [[l[0],l[3]] for k,l in zip(molecule.torsion_keys,molecule.torsion_list) if reftor_name == k]

In [23]:
input_file_name = "force-fields/forcefield_lammps.toml"
with open(input_file_name) as toml_file:
    ff = toml.load(toml_file)

def get_name(name):
    if name[0] == "c":
        return name[1]
    else:
        return name[0]
    
#get_name("CH3_alkane"), get_name("OH_EOH"), get_name("cH_EOH")

playmol_atom_names = [ get_name(mn) for mn in molecule.atom_names]
playmol_bond_names = [ [ get_name( molecule.atom_names[mi]) for mi in bl ] for bl in molecule.bond_list]

In [24]:
#Schreibe File für Playmol mit force_field paras

renderdict = {}

#Bestimmte die UA die im Molekül vorkommen und schreibe dict mit FF Paras
nonbonded                  = molecule.map_molecule( molecule.unique_atom_keys, ff["atoms"] )
renderdict["nonbonded"]    = list( zip( molecule.unique_atom_keys, nonbonded  ) ) 

#Liste alle bonds mit zugehörigen FF Paras
bonds                      = molecule.map_molecule( molecule.unique_bond_keys , ff["bonds"] )
renderdict["bonds"]        = list( zip(molecule.unique_bond_names, bonds) )

#Liste alle angles mit zugehörigen FF Paras      
angles                     = molecule.map_molecule( molecule.unique_angle_keys , ff["angles"] )
renderdict["angles"]       = list( zip(molecule.unique_angle_names, angles) )

#Liste alle torsions mit zugehörigen FF Paras
torsions                   = molecule.map_molecule( molecule.unique_torsion_keys , ff["torsions"] )
renderdict["torsions"]     = list( zip(molecule.unique_torsion_names, torsions) )

#Generiere Playmol input style für bonds,angles,torsions via templates (using jinja2)

with open("templates/template.playmol") as file_:
    template = Template(file_.read())
rendered = template.render( rd=renderdict ) #Textformat wie in template
dummy = "inout_playmol/"+name+".playmol"

with open(dummy, "w") as fh:
    fh.write(rendered) 
    
atom_numbers = list( np.arange(molecule.atom_number) +1 )

In [25]:
#Add Special Bond via bond_style table command
lammps_molecule = moleculegraph.molecule(molecule_graphstring)

if special_bonds:
    
    #Get local dipoles in the molecule
    #todo: check if this works for every molecule

    flag = 0
    dipol_list = []
    for i,(atom,no) in enumerate(zip(molecule.atom_names,molecule.atom_numbers)):
            
        if (ff["atoms"][atom]["charge"] != 0 and flag == 0):
            dipol = []
            chrg = 0
            chrg += ff["atoms"][atom]["charge"]
            flag = 1
            dipol.append(no)

        elif(ff["atoms"][atom]["charge"] != 0 and flag == 1):

            #Check if next atom is bonded to the previous one

            atom_no1 = molecule.atom_numbers[i]
            atom_no2 = molecule.atom_numbers[i-1]
            distance = molecule.get_distance(atom_no1,atom_no2)

            if distance == 1:
                chrg += ff["atoms"][atom]["charge"]
                dipol.append(no)

            #Check if the total charge is 0, then its a local dipol

                if np.round(chrg,3) == 0:
                    dipol_list.append(dipol)
                    flag = 0
    # make special_bonds
    # TO DO: the terms number and index might be messed up... not important for diols...
    special_bond_indexes = np.array( np.meshgrid( dipol_list[0], dipol_list[1] ) ).T.reshape(-1, 2)
    special_bond_names = molecule.atom_names[special_bond_indexes]
    special_bond_keys = [ "[special]"+moleculegraph.make_graph(moleculegraph.sort_force_fields(x)) for x in special_bond_names ]
    special_bond_keys_unique, uidx = np.unique(special_bond_keys, return_index=True)
    special_bond_names_unique = special_bond_names[uidx]
    
    # delete unnecessary standard bonds 
    all_bonds = np.concatenate( (molecule.bond_list, special_bond_indexes ))
    _ ,idxx, counts = np.unique(all_bonds ,axis=0,return_index=True, return_counts=True)
    del_idx  = idxx[ np.where(counts>1) ]
    lammps_molecule.bond_list   = np.concatenate( (np.delete(lammps_molecule.bond_list, del_idx, axis=0), special_bond_indexes ),axis=0 )
    lammps_molecule.bond_names  = np.concatenate( (np.delete(lammps_molecule.bond_names, del_idx, axis=0), special_bond_names ),axis=0 )
    lammps_molecule.bond_keys   = np.concatenate( (np.delete(lammps_molecule.bond_keys, del_idx, axis=0),  special_bond_keys ),axis=0 )
    
    # build new unique bonds info
    dummy = moleculegraph.molecule_utils.unique_sort(lammps_molecule.bond_keys,return_inverse=True)
    lammps_molecule.unique_bond_keys, lammps_molecule.unique_bond_indexes, lammps_molecule.unique_bond_inverse = dummy 
    lammps_molecule.unique_bond_names  = lammps_molecule.bond_names[ lammps_molecule.unique_bond_indexes ]
    lammps_molecule.unique_bond_numbers = lammps_molecule.bond_list[ lammps_molecule.unique_bond_indexes ]    

    table_path = name +".table"

    for ( bkey, bname ) in zip(special_bond_keys_unique, special_bond_names_unique):
        dict1             = {}
        dict1["list"]     = list(bname)
        dict1["p"]        = [ bkey , table_path ]
        dict1["style"]    = "table"
        dict1["type"]     = -1

        ff["bonds"][bkey] = dict1
    
else:
    print("nothing to be done")

In [27]:
moldict = {}
moldict["atoms"]          = list(zip( atom_numbers , molecule.atom_names, playmol_atom_names) )
atoms_dummy              = molecule.map_molecule( molecule.atom_names, ff["atoms"] ) #liste mit atoms und ihre FF Paras
mass = np.sum([ n["mass"] for n in atoms_dummy ])                                    #Gesamtgewicht des Moleküls
moldict["charges"]       = list(zip( atom_numbers , atoms_dummy, playmol_atom_names))#liste mit Atomnummern und dazugehörigen FF Paras
moldict["bonds"] = list(zip( molecule.bond_list+1, playmol_bond_names))
moldict["name"]  = name


folder = "inout_playmol/"

shutil.copyfile("molecule_xyz/"+name+"_playmol.xyz", folder+name+"_playmol.xyz")

maindir = os.getcwd()
for i,rho in enumerate(densities):
    with open("templates/template.mol") as file_:
        template = Template(file_.read())
    rendered = template.render( rd=moldict, name=name,rho=str(rho/1000),i=str(i),nmols=nmols )
    dummy = folder+name+"_"+str(i)+".mol"
    with open(dummy, "w") as fh:
        fh.write(rendered) 
    
    build = True
    #build = False
    if build:
        os.chdir(folder)
        console = "playmol -i "+name+"_"+str(i)+".mol"
        log = os.system( console )
        os.chdir(maindir)
        print(log)
        print("DONE: "+console)

    atom_count  = 0
    bond_count  = 0
    angle_count = 0
    torsion_count    = 0
    molecule_count   = 0
    coordinate_count = 0

    lmp_atom_list = []
    lmp_bond_list = []
    lmp_angle_list = []
    lmp_torsion_list = []
    
    dummy = folder+name+"_"+str(i)+".xyz"
    coordinates = moleculegraph.general_utils.read_xyz(dummy)
    
    boxlen = (mass/rho/1000/Avogadro*10**30*nmols)**(1/3)/2 #Berechne Boxvolumen V=m/rho und daraus Kantenlänge L (genauer L/2)
    box = [ -boxlen, boxlen] # definiere in jeder Dimension die Boxränder (+L/2,L/2)
    
    renderdict = {}
    renderdict["box_x"] = box
    renderdict["box_y"] = box
    renderdict["box_z"] = box

    atom_paras   = molecule.map_molecule( molecule.unique_atom_names, ff["atoms"] ) #ff paras der UA
    charges      = molecule.map_molecule( molecule.atom_names, ff["atoms"] )
    number_of_atoms = len( molecule.atom_names )
    total_number_of_atoms = int( number_of_atoms*nmols )     
    atoms_running_number = molecule.unique_atom_inverse +1  
    atom_numbers = molecule.atom_numbers+1  
    renderdict["atom_paras"] = zip( atom_numbers, atom_paras )
    renderdict["atom_type_number"] = len(molecule.unique_atom_indexes)

    bond_paras   = lammps_molecule.map_molecule( lammps_molecule.unique_bond_keys, ff["bonds"] )
    number_of_bonds = len( lammps_molecule.bond_keys )
    total_number_of_bonds = int( number_of_bonds*nmols )     
    bonds_running_number = lammps_molecule.unique_bond_inverse +1
    bond_numbers = lammps_molecule.bond_list+1
    bond_numbers_ges = np.sort(np.unique(bonds_running_number))
    renderdict["bond_styles"] = list(np.unique([p["style"] for n,p in zip( bond_numbers_ges, bond_paras) ] ) )
    renderdict["bond_paras"] = zip( bond_numbers_ges, bond_paras )
    renderdict["bond_type_number"] = len(lammps_molecule.unique_bond_indexes)
    
    angle_paras   = molecule.map_molecule( molecule.unique_angle_keys, ff["angles"] )
    number_of_angles = len( molecule.angle_keys )
    total_number_of_angles = int( number_of_angles*nmols ) 
    angles_running_number = molecule.unique_angle_inverse +1
    angle_numbers = molecule.angle_list+1
    
    renderdict["angle_styles"] = list(np.unique([p["style"] for n,p in zip(  np.sort(np.unique(angles_running_number)), angle_paras ) ]))
    renderdict["angle_paras"] = zip(  np.sort(np.unique(angles_running_number)), angle_paras )
    renderdict["angle_type_number"] = len(molecule.unique_angle_indexes)
    
    torsion_paras   = molecule.map_molecule( molecule.unique_torsion_keys, ff["torsions"] )
    number_of_torsions = len( molecule.torsion_keys )
    total_number_of_torsions = int( number_of_torsions*nmols ) 
    torsions_running_number = molecule.unique_torsion_inverse +1
    torsion_numbers = molecule.torsion_list +1
    
    renderdict["torsion_styles"] = list(np.unique([p["style"] for n,p in zip(  np.sort(np.unique(torsions_running_number)), torsion_paras ) ]))
    renderdict["torsion_paras"] = zip(  np.sort(np.unique(torsions_running_number)), torsion_paras )
    renderdict["torsion_type_number"] = len(molecule.unique_torsion_indexes)
    
    #Schreibe LAMMPS input file (using template)
    
    for _ in range(nmols):
        
        for atomtype , charge in zip( atoms_running_number , charges ):
            atom_count += 1
    
            line = [ atom_count, molecule_count+1, atomtype, charge["charge"], 
                    *coordinates[coordinate_count]["xyz"],  coordinates[coordinate_count]["atom"]  ]
            lmp_atom_list.append(line)
            coordinate_count += 1
        
        for bondtype ,bond in zip( bonds_running_number , bond_numbers ):
            bond_count += 1
            #Sorgt dafür dass die Atomnummer entsprechend des betrachteten Moleküls richtig ist
            dummy = bond + molecule_count*molecule.atom_number
            line = [ bond_count, bondtype, *dummy, *[ "A"+str(x) for x in bond ] ]
            lmp_bond_list.append(line)
        
        for angletype ,angle in zip( angles_running_number , angle_numbers ):
            angle_count += 1
            dummy = angle + molecule_count*molecule.atom_number
            line = [ angle_count, angletype, *dummy, *[ "A"+str(x) for x in angle ]  ]
            lmp_angle_list.append(line)

        for torsiontype ,torsion in zip( torsions_running_number , torsion_numbers ):
            torsion_count += 1
            dummy = torsion + molecule_count*molecule.atom_number
            dummy = np.flip( dummy ) # (?)
            torsion = np.flip( torsion)  # (?)
            line = [ torsion_count, torsiontype, *dummy, *[ "A"+str(x) for x in torsion ]   ]
            lmp_torsion_list.append(line)
            
        molecule_count +=1  
    
    renderdict["atoms"]    = lmp_atom_list
    renderdict["bonds"]    = lmp_bond_list
    renderdict["angles"]   = lmp_angle_list
    renderdict["torsions"] = lmp_torsion_list

    renderdict["atom_number"]    = atom_count
    renderdict["bond_number"]    = bond_count
    renderdict["angle_number"]   = angle_count
    renderdict["torsion_number"] = torsion_count

    in_name = group+ "/" + name+"/"+name+"_"+str(i)+".lmp"
   
    with open("templates/template.lmp") as file_:
        template = Template(file_.read())
    rendered = template.render( rd = renderdict )
    
    with open(in_name, "w") as fh:
        fh.write(rendered)

Playmol (Version: 04 Mar 2021)
Reading file 1.2-Ethanediol_0.mol...
Adding variable rho with parameters 1.0753316421595096
Adding variable seed with parameters 1337
Adding variable nmols with parameters 500
Adding variable x with parameters 0
Including file 1.2-Ethanediol.playmol
Adding atom type cH_alcohol with parameters mie/cut 0.0 1.0 12.0 6.0
Adding diameter cH_alcohol with parameters 1.0
Adding mass cH_alcohol with parameters 1.0
Adding charge cH_alcohol with parameters 0.404
Adding atom type OH_alcohol with parameters mie/cut 0.1673 3.035 12.0 6.0
Adding diameter OH_alcohol with parameters 3.035
Adding mass OH_alcohol with parameters 17.007
Adding charge OH_alcohol with parameters -0.65
Adding atom type CH2_alcohol with parameters mie/cut 0.1673 3.842 14.0 6.0
Adding diameter CH2_alcohol with parameters 3.842
Adding mass CH2_alcohol with parameters 14.027
Adding charge CH2_alcohol with parameters 0.246
Adding bond type OH_alcohol cH_alcohol with parameters harmonic 1000.0 0.97
A

Converged!
Writing data to file 1.2-Ethanediol_1.lmp in lmp/models format...
3 atom types
3 bond types
3000 atoms
2500 bonds
Writing data to file 1.2-Ethanediol_1.xyz in xyz format...
Writing data to file 1.2-Ethanediol_1.log in summary format...
--------------------------------------------------------------------------------
SUMMARY
--------------------------------------------------------------------------------
Specified:
    3 atom types.
    3 bond types.
    2 angle types.
    2 dihedral types.
    6 atoms.
    5 bonds.

Detected:
    1 molecule.

Effectively usable:
    3 atom types.
    3 bond types.

Molecule[1]:
- Amount: 500
- Mass: 64.0680008
- Charge: 0.00000000
- Number of atoms: 6
- Atoms: H1 O2 C3 C4 O5 H6

System:
- Molecules: 500
- Total mass: 32034.0000
- Box lengths: 36.7685089 36.7685089 36.7685089
- Box density: 0.644440949
- Residual charge: 0.00000000
--------------------------------------------------------------------------------
0
DONE: playmol -i 1.2-Ethanedio

Converged!
Writing data to file 1.2-Ethanediol_3.lmp in lmp/models format...
3 atom types
3 bond types
3000 atoms
2500 bonds
Writing data to file 1.2-Ethanediol_3.xyz in xyz format...
Writing data to file 1.2-Ethanediol_3.log in summary format...
--------------------------------------------------------------------------------
SUMMARY
--------------------------------------------------------------------------------
Specified:
    3 atom types.
    3 bond types.
    2 angle types.
    2 dihedral types.
    6 atoms.
    5 bonds.

Detected:
    1 molecule.

Effectively usable:
    3 atom types.
    3 bond types.

Molecule[1]:
- Amount: 500
- Mass: 64.0680008
- Charge: 0.00000000
- Number of atoms: 6
- Atoms: H1 O2 C3 C4 O5 H6

System:
- Molecules: 500
- Total mass: 32034.0000
- Box lengths: 37.2910576 37.2910576 37.2910576
- Box density: 0.617727697
- Residual charge: 0.00000000
--------------------------------------------------------------------------------
0
DONE: playmol -i 1.2-Ethanedio

Converged!
Writing data to file 1.2-Ethanediol_5.lmp in lmp/models format...
3 atom types
3 bond types
3000 atoms
2500 bonds
Writing data to file 1.2-Ethanediol_5.xyz in xyz format...
Writing data to file 1.2-Ethanediol_5.log in summary format...
--------------------------------------------------------------------------------
SUMMARY
--------------------------------------------------------------------------------
Specified:
    3 atom types.
    3 bond types.
    2 angle types.
    2 dihedral types.
    6 atoms.
    5 bonds.

Detected:
    1 molecule.

Effectively usable:
    3 atom types.
    3 bond types.

Molecule[1]:
- Amount: 500
- Mass: 64.0680008
- Charge: 0.00000000
- Number of atoms: 6
- Atoms: H1 O2 C3 C4 O5 H6

System:
- Molecules: 500
- Total mass: 32034.0000
- Box lengths: 41.9871559 41.9871559 41.9871559
- Box density: 0.432774633
- Residual charge: 0.00000000
--------------------------------------------------------------------------------
0
DONE: playmol -i 1.2-Ethanedio

In [28]:
#Bestimmte die FF Parameter für jede Pair interaction (auch welche die nicht vorkommen)

atoms_dummy = molecule.map_molecule( molecule.unique_atom_keys, ff["atoms"] )
atom_numbers = np.arange(len( molecule.unique_atom_keys )) +1

pair_interactions = []
for i,iatom in zip(atom_numbers, atoms_dummy):
    for j,jatom in zip(atom_numbers[i-1:], atoms_dummy[i-1:]):
        dummy = {}
        dummy["i"] = i
        dummy["j"] = j
        dummy["sigma"]   = round( ( iatom["sigma"] + jatom["sigma"] ) / 2 , 4)
        dummy["epsilon"] = round( np.sqrt( iatom["epsilon"] * jatom["epsilon"] ) , 4)
        dummy["m"]       = ( iatom["m"] + jatom["m"] ) / 2
        dummy["charge"] = round( iatom["charge"] * jatom["charge"] , 4)
        pair_interactions.append(dummy)
        
#in_template = "../templates/in.npt_equilibration_template"

#with open(in_template) as file_:
#    template = Template(file_.read())
#rendered = template.render( pairs = pair_interactions )
#out = in_template.split("/")[-1].replace("_template","")
#with open(out, "w") as fh:
#    fh.write(rendered)

#Dynamisches reinschreiben der verwendeten Styles
style = {}
try:
    bond_styles = list(np.unique([[a["style"] for a in bond_paras]+[a["bond_key"] for a in special]]))
except:
    bond_styles = list(np.unique([a["style"] for a in bond_paras]))
    
if len(bond_styles) >1:
    bond_styles = ["hybrid"] + bond_styles

dihedral_styles = list(np.unique([a["style"] for a in torsion_paras]))
if len(dihedral_styles) >1:
    dihedral_styles = ["hybrid"] + dihedral_styles

style["bond"]     = bond_styles
style["angle"]    = ["harmonic"]
style["dihedral"] = dihedral_styles

for i,_ in enumerate(densities):
    in_name = name+"_"+str(i)+".lmp"
    in_template = "../templates/in.test_template"

    #Nutze Template um lamps system input zu schreiben
    #Im Template werden pair_style etc vorgegeben

    with open(in_template) as file_:
        template = Template(file_.read())
    rendered = template.render( pairs = pair_interactions, style = style,path=in_name )
    out = "../"+group+ "/" + name+"/"+in_template.split("/")[-1].replace("_template","")+str(i)
    #print(rendered)
    with open(out, "w") as fh:
        fh.write(rendered)   
        
    in_template = "templates/in.test_template"

#Nutze Template um ff input zu schreiben
#Im Template werden pair_style etc vorgegeben
in_template = "templates/ff_template.ff"
with open(in_template) as file_:
    template = Template(file_.read())
rendered = template.render( pairs = pair_interactions, style = style,path=in_name )
out = "../"+group+ "/" + name+"/"+in_template.split("/")[-1].replace("_template","")
#print(rendered)
with open(out, "w") as fh:
    fh.write(rendered)           

FileNotFoundError: [Errno 2] No such file or directory: '../templates/in.test_template'